# install dependecnies

In [1]:
!pip install -q datasets
!pip install -q transformers
!pip install -q umap-learn
!pip install -q bertviz
!pip install -q accelerate
!pip install -q seqeval
!pip install -q tqdm
!pip install -q scikit-learn 
!pip install -q transformers
!pip install -q huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new 

In [2]:
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [4]:
from huggingface_hub import notebook_login
notebook_login()

# Imports

In [5]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter

# Hyperparameters

In [6]:
file_id = '1w3IQMK3PmXH-Bq6Lt_P8wxGPvr5olrZT'
destination = 'pfam.zip'
hf_name = "t5-small"
n_families_of_interest = 1000
data_dirpath = "pfam"
aminoacid_separate_by = ""
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

# Utils

In [7]:
def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

In [8]:
# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm.tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")

# Get the dataset

In [9]:
download_file_from_google_drive(file_id, destination)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(".")
!mv random_split pfam

written : 468.781 MB: : 15780it [00:11, 1368.57it/s]




saved all the data to pfam.zip. total size : 493.095 MB


# Read data

In [10]:
def read_all_shards(partition='dev', data_dir = data_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    
    return pd.concat(shards)

def read_all_data_initial():
  global train, test, dev, all_train_ds_size, all_test_ds_size, all_dev_ds_size

  test = read_all_shards('test')
  dev = read_all_shards('dev')
  train = read_all_shards('train')

  partitions = {'test': test, 'dev': dev, 'train': train}
  for name, df in partitions.items():
      print('Dataset partition "%s" has %d sequences' % (name, len(df)))

  all_train_ds_size = len(train)
  all_test_ds_size = len(test)
  all_dev_ds_size = len(dev)

  train.reset_index(inplace=True, drop=True)
  dev.reset_index(inplace=True, drop=True)
  test.reset_index(inplace=True, drop=True)

def get_cumulative(data):
    counter = Counter(data['family_accession'])
    print(f"how many labels : {len(counter)}")
    
    datasetSize = len(data)
    xs = []
    x_labels = []
    ys = []

    t = 0
    cumulative = []

    for i,(x, y) in  enumerate(counter.most_common()):
        xs.append(i)
        x_labels.append(x)
        ys.append(y)
        t += y / datasetSize
        cumulative.append(t)
    return cumulative


# EXECUTION CODE
print('Available dataset partitions: ', os.listdir(data_dirpath))
read_all_data_initial()
cumulative = get_cumulative(train)
print(f"{n_families_of_interest} classes is {100 * round( cumulative[n_families_of_interest-1],3)} portion of training data")

familiesOfInterest = train.family_accession.value_counts()[:n_families_of_interest]

mask = train.family_accession.isin(familiesOfInterest.index.values)
train = train.loc[mask,:]

mask = dev.family_accession.isin(familiesOfInterest.index.values)
dev = dev.loc[mask,:]

mask = test.family_accession.isin(familiesOfInterest.index.values)
test = test.loc[mask,:]


################################################################################
train_seq = train['sequence']
dev_seq = dev['sequence']
test_seq = test['sequence']

################################################################################
train_sentences = train_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
validation_sentences = dev_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
test_sentences = test_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))

################################################################################
train_labels = train['family_accession'].apply(lambda x: x.split('.')[0])
validation_labels = dev['family_accession'].apply(lambda x: x.split('.')[0])
test_labels = test['family_accession'].apply(lambda x: x.split('.')[0])

Available dataset partitions:  ['train', 'test', 'random_split', 'dev']
Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
how many labels : 17929
1000 classes is 40.400000000000006 portion of training data


In [11]:
sequence = "RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL"

train_labels[train_sentences == sequence]

0    PF00365
Name: family_accession, dtype: object

In [12]:
train_sentences[train_sentences == sequence]

0    RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGD...
Name: sequence, dtype: object

In [13]:
train_labels

0          PF00365
4          PF01237
7          PF13277
9          PF00831
10         PF00560
            ...   
1086720    PF13649
1086724    PF01890
1086729    PF13004
1086735    PF04066
1086738    PF14748
Name: family_accession, Length: 439493, dtype: object

In [14]:
pfam_famililes = set(train_labels.values)

with open("pfam_families.txt", "w") as f:
  for pfam_family in pfam_famililes:
    f.write(pfam_family + "\n")

In [15]:
labels = set()

for v in tqdm.tqdm(train_labels):
  labels.add(v)

label_to_id = {k:v for v,k in enumerate(labels)}

print()
print(len(labels))

100%|██████████| 439493/439493 [00:00<00:00, 2535552.91it/s]


1000


In [16]:
import json

with open("pfam_to_id.json", "w") as f:
  json.dump(label_to_id, f)

In [17]:
train_labels

0          PF00365
4          PF01237
7          PF13277
9          PF00831
10         PF00560
            ...   
1086720    PF13649
1086724    PF01890
1086729    PF13004
1086735    PF04066
1086738    PF14748
Name: family_accession, Length: 439493, dtype: object

In [18]:
train_labels_as_int = train_labels.map(label_to_id)
val_labels_as_int = validation_labels.map(label_to_id)

In [19]:
len(set(train_labels_as_int.values))

1000

In [20]:
from datasets import Dataset
dataset = Dataset.from_dict({"sentence": train_sentences, "labels": train_labels_as_int})
dataset_val = Dataset.from_dict({"sentence": validation_sentences, "labels": val_labels_as_int})

In [21]:
dataset

Dataset({
    features: ['sentence', 'labels'],
    num_rows: 439493
})

In [22]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training

tokenizer.train_from_iterator(iter(train_sentences), vocab_size=vocab_size, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=max_length)

In [23]:
!mkdir {tokenizer_folder}
tokenizer.save_model(tokenizer_folder)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['PROTNAME_tok/vocab.json', 'PROTNAME_tok/merges.txt']

In [24]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint) 
print(f"datapoint : {datapoint}, len({len(datapoint)})")
print(encoded)

datapoint : RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL, len(276)
Encoding(num_tokens=278, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [25]:
print(encoded.ids)

[0, 54, 45, 43, 45, 49, 56, 55, 43, 43, 40, 37, 52, 43, 49, 50, 48, 37, 45, 54, 37, 58, 37, 54, 47, 37, 48, 55, 55, 43, 48, 41, 37, 61, 43, 45, 50, 61, 43, 42, 37, 43, 48, 58, 37, 43, 40, 45, 44, 41, 42, 47, 37, 37, 40, 48, 40, 40, 49, 58, 55, 53, 43, 43, 56, 49, 48, 61, 55, 37, 54, 61, 52, 41, 42, 37, 53, 41, 41, 55, 53, 48, 47, 43, 45, 41, 53, 48, 47, 47, 42, 43, 45, 40, 37, 48, 58, 58, 45, 43, 43, 40, 43, 55, 61, 44, 43, 37, 48, 54, 48, 56, 41, 44, 43, 61, 50, 56, 45, 43, 48, 52, 43, 56, 45, 40, 50, 40, 45, 52, 42, 56, 40, 42, 56, 45, 43, 42, 40, 56, 37, 48, 50, 56, 37, 58, 40, 37, 45, 40, 47, 45, 54, 40, 56, 37, 47, 55, 44, 53, 54, 58, 42, 37, 58, 53, 58, 49, 43, 54, 50, 37, 37, 40, 45, 37, 48, 59, 37, 43, 58, 37, 55, 43, 37, 40, 37, 58, 45, 37, 52, 43, 42, 40, 61, 40, 58, 41, 37, 45, 37, 50, 47, 48, 47, 47, 50, 54, 37, 50, 43, 47, 40, 61, 43, 45, 45, 58, 45, 37, 41, 43, 40, 37, 50, 55, 40, 37, 37, 52, 41, 42, 45, 40, 53, 48, 47, 53, 61, 43, 40, 42, 40, 37, 54, 37, 56, 58, 45, 43, 

In [26]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [27]:
# Check that we have a GPU
!nvidia-smi
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Apr 24 11:09:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0 Off |                  Off |
|  0

True

In [28]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=max_length, padding=True)
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/439493 [00:00<?, ? examples/s]

Map:   0%|          | 0/54378 [00:00<?, ? examples/s]

In [43]:
from transformers import BertConfig
from transformers import BertForSequenceClassification

config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=max_length+2,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=8,
    type_vocab_size=1,
    num_labels=n_families_of_interest+1,
    label2idx=label_to_id
)

model = BertForSequenceClassification(config=config)

In [46]:
print(model.config.label2idx)

{'PF17802': 0, 'PF05013': 1, 'PF18821': 2, 'PF18075': 3, 'PF04039': 4, 'PF06130': 5, 'PF02654': 6, 'PF00163': 7, 'PF00719': 8, 'PF06165': 9, 'PF17921': 10, 'PF00376': 11, 'PF02466': 12, 'PF00237': 13, 'PF05649': 14, 'PF06094': 15, 'PF01369': 16, 'PF01649': 17, 'PF01975': 18, 'PF07503': 19, 'PF00731': 20, 'PF03334': 21, 'PF01985': 22, 'PF11760': 23, 'PF01268': 24, 'PF04055': 25, 'PF08712': 26, 'PF00233': 27, 'PF02537': 28, 'PF04122': 29, 'PF03619': 30, 'PF08148': 31, 'PF02739': 32, 'PF03323': 33, 'PF04237': 34, 'PF10436': 35, 'PF06421': 36, 'PF04253': 37, 'PF01657': 38, 'PF02545': 39, 'PF02811': 40, 'PF00396': 41, 'PF00557': 42, 'PF07884': 43, 'PF00312': 44, 'PF03862': 45, 'PF01339': 46, 'PF17757': 47, 'PF09269': 48, 'PF03449': 49, 'PF02686': 50, 'PF02777': 51, 'PF01867': 52, 'PF17202': 53, 'PF01193': 54, 'PF01904': 55, 'PF08486': 56, 'PF06965': 57, 'PF14805': 58, 'PF16124': 59, 'PF14403': 60, 'PF02881': 61, 'PF13565': 62, 'PF02625': 63, 'PF02436': 64, 'PF13793': 65, 'PF08393': 66, 'PF0

In [47]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(261, 128, padding_idx=0)
      (position_embeddings): Embedding(514, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, ele

In [48]:
model.num_parameters() / 1024 / 1024

1.8669214248657227

In [49]:
from datasets import load_dataset, load_metric

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [51]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=5e-4,
    num_train_epochs=10,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    push_to_hub=True,
    hub_model_id="mlewand/PROTBERT-tiny",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/workspace/PROTNAME is already a clone of https://huggingface.co/mlewand/PROTBERT-tiny. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.685800,3.452489,0.187245,0.109976
2,3.440100,3.175666,0.239398,0.163652
3,3.205800,3.005582,0.264795,0.194011
4,3.049600,2.804687,0.312847,0.237627
5,2.937200,2.730087,0.316672,0.243396
6,2.840000,2.626378,0.343963,0.271696
7,2.759600,2.541703,0.368421,0.297097
8,2.688800,2.499723,0.376623,0.309842


KeyboardInterrupt: 

In [54]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    push_to_hub=True,
    hub_model_id="mlewand/PROTBERT-tiny",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/workspace/PROTNAME is already a clone of https://huggingface.co/mlewand/PROTBERT-tiny. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.150300,0.799259,0.804241,0.795231
2,0.892100,0.631554,0.849314,0.843460
3,0.695900,0.476182,0.888852,0.885345
4,0.553900,0.386687,0.910718,0.908633
5,0.457100,0.338941,0.923020,0.921561


TrainOutput(global_step=17170, training_loss=0.7498414721125145, metrics={'train_runtime': 2342.788, 'train_samples_per_second': 937.97, 'train_steps_per_second': 7.329, 'total_flos': 1.2416350270311936e+16, 'train_loss': 0.7498414721125145, 'epoch': 5.0})

In [ ]:
training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

In [73]:
model.config.label2id = label_to_id

In [78]:
model.config.id2label = {v:k for k,v in label_to_id.items()}
model.config.id2label[1000] = "unk"

In [81]:
model.push_to_hub("mlewand/PROTBERT-tiny")

CommitInfo(commit_url='https://huggingface.co/mlewand/PROTBERT-tiny/commit/87cbcabb67189ffbf6ba54101e1ef818736baaf5', commit_message='Upload BertForSequenceClassification', commit_description='', oid='87cbcabb67189ffbf6ba54101e1ef818736baaf5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint) 
print(f"datapoint : {datapoint}, len({len(datapoint)})")

datapoint : RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL, len(276)


In [82]:
from transformers import pipeline
pipe = pipeline(
    'text-classification',
    model=model.to("cpu"),
    tokenizer=tokenizer
)

In [83]:
pipe("RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL")

[{'label': 'PF00365', 'score': 0.9787689447402954}]

In [90]:
import requests
import json
import os

In [93]:
with open("pfam_families1.txt", "w") as f:
    f.write("\n".join(label_to_id.keys()))

In [96]:
pfam_ids = list(label_to_id.keys())

In [99]:
def get_description(id):
    response = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/pfam/{id}")
    return response.json()

def get_description_mock(id):
    return {"metadata" : {"description" : "mock"}}

def collate_all_families(pfam_familiy_ids, mock=True):
    dataset = {}
    for pfam_id in tqdm.tqdm(pfam_familiy_ids):
        if mock:
            dataset[pfam_id] = get_description_mock(pfam_id)
        else:
            dataset[pfam_id] = get_description(pfam_id)    

    return dataset

with open(os.path.join(".", "pfam_data.json"), "w") as f:
    json.dump(collate_all_families(pfam_ids, mock=False), f)

100%|██████████| 1000/1000 [05:32<00:00,  3.00it/s]


In [102]:
with open(os.path.join("pfam_data.json")) as f:
    data = json.load(f)

In [106]:
pfam_id_to_number = {v:k for k, v in label_to_id.items()}

In [115]:
BERT_model_idx2label = {}
for family_id in tqdm.tqdm(pfam_ids):
    name = data[family_id]["metadata"]["name"]["name"]
    description = data[family_id]["metadata"]["description"]

    # print(f"{family_id} -> {name}, {description}")
    entry = f"{name}"
    if description is not None:
        desc = description[0].replace("<p>", "").replace("</p>", "")
        entry = f"{family_id}\n{name}\n{desc}"
    BERT_model_idx2label[label_to_id[family_id]] = entry

100%|██████████| 1000/1000 [00:00<00:00, 257051.17it/s]


In [120]:
model.config.id2label = BERT_model_idx2label
model.config.id2label[1000] = "unk"

In [129]:
model.push_to_hub("mlewand/PROTBERT-tiny")

CommitInfo(commit_url='https://huggingface.co/mlewand/PROTBERT-tiny/commit/32d1283f41ddf69883ef7ace40785425c658810a', commit_message='Upload BertForSequenceClassification', commit_description='', oid='32d1283f41ddf69883ef7ace40785425c658810a', pr_url=None, pr_revision=None, pr_num=None)

In [121]:
from transformers import pipeline
pipe = pipeline(
    'text-classification',
    model=model.to("cpu"),
    tokenizer=tokenizer
)

In [128]:
pipe("SLKFLNFAQNEFNGSIPESV")

[{'label': 'PF00560\nLeucine Rich Repeat\nCAUTION: This Pfam may not find all Leucine Rich Repeats in a protein.  Leucine Rich Repeats are short sequence motifs present in a number of proteins with diverse functions and cellular locations.  These repeats are usually involved in protein-protein interactions. Each Leucine Rich Repeat is composed of a beta-alpha unit. These units form elongated non-globular structures. Leucine Rich Repeats are often flanked by cysteine rich domains.',
  'score': 0.9977093935012817}]

In [127]:
train_labels[train_sentences == "SLKFLNFAQNEFNGSIPESV"]

10    PF00560
Name: family_accession, dtype: object

In [134]:
from transformers import pipeline
pipe = pipeline(
    'text-classification',
    model="mlewand/PROTBERT-tiny",
    tokenizer="mlewand/PROTBERT-tiny"
)
print(pipe("SLKFLNFAQNEFNGSIPESV")[0]["label"])

PF00560
Leucine Rich Repeat
CAUTION: This Pfam may not find all Leucine Rich Repeats in a protein.  Leucine Rich Repeats are short sequence motifs present in a number of proteins with diverse functions and cellular locations.  These repeats are usually involved in protein-protein interactions. Each Leucine Rich Repeat is composed of a beta-alpha unit. These units form elongated non-globular structures. Leucine Rich Repeats are often flanked by cysteine rich domains.
